In [ ]:
import json
import pandas as pd
from tqdm import tqdm

AI_TOOL = 'chatgpt' # copilot/chatgpt

# valid repo list
all_repos = pd.read_csv('./repos.csv')
print(f'Repos elements: {len(all_repos)}')
filter_by = 'name' 
FILTERD_VALUES = [_.lower() for _ in all_repos[filter_by].to_list()]
print(f"repo element example: {FILTERD_VALUES[42]}")

In [3]:
def load_data(path_data):
    elements = []
    with open(path_data) as f:
        for line in f.readlines():
            line = line.strip()
            elements.append(line)
    return elements

def filter_elements(elements_to_filter, type):
    discarded_elements = 0
    valid_elements = []
    for i in tqdm(range(len(elements_to_filter))):
        elem = elements_to_filter[i]
        if not elem.endswith('}'): 
            print('FALG!')
            elem += '}'

        elem = json.loads(elem)
        if type == 'pr':
            repo_name = '/'.join(elem["repository_url"].split('/')[-2:])

        elif type == 'commit':
            repo_name = '/'.join(elem["url"].split('/commits/')[0].split('/')[-2:])

        elif type == 'issue':
            repo_name = '/'.join(elem["url"].split('/issues/')[0].split('/')[-2:])

        if repo_name.lower() not in FILTERD_VALUES: 
            discarded_elements += 1
        else:
            valid_elements.append(elem)
    return valid_elements, discarded_elements

def save_data(filename, elements):
    with open(filename, 'w') as file:
        for elem in elements:
            file.write(json.dumps(elem) + '\n')

In [ ]:
# pr
print('\n##### PR\n')

pr_path_data = 'prs-' + AI_TOOL + '.jsonl' # put here the correct path to the file
pr_elements = load_data(pr_path_data)
print(f'PR elements: {len(pr_elements)}')

valid_pr, discarded_pr = filter_elements(pr_elements, 'pr')
print(f'Discarded PRs: {discarded_pr}')
print(f'Valid PRs: {len(valid_pr)}')

save_data('valid_prs_' + AI_TOOL + '.jsonl', valid_pr)

# commit
print('\n##### COMMIT\n')

commit_path_data = 'commits-' + AI_TOOL + '.jsonl' # put here the correct path to the file
commit_elements = load_data(commit_path_data)
print(f'Commit elements: {len(commit_elements)}')

valid_commit, discarded_commit = filter_elements(commit_elements, 'commit')
print(f'Discarded commits: {discarded_commit}')
print(f'Valid commits: {len(valid_commit)}')

save_data('valid_commits_' + AI_TOOL + '.jsonl', valid_commit)

# issue
print('\n##### ISSUE\n')

issue_path_data = 'issues-' + AI_TOOL + '.jsonl' # put here the correct path to the file
issue_elements = load_data(issue_path_data)
print(len(issue_elements))
print(f'Issue elements: {len(issue_elements)}')

valid_issue, discarded_issue = filter_elements(issue_elements, 'issue')
print(f'Discarded issues: {discarded_issue}')
print(f'Valid issues: {len(valid_issue)}')

save_data('valid_issues_' + AI_TOOL + '.jsonl', valid_issue)